# SMBO Gaussian Process - New York City Taxi

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import seaborn as sns
import pandas as pd

import xgboost as xgb
from bayes_opt import BayesianOptimization
# bayesian-optimization
from sklearn.metrics import r2_score

sns.set()

## Chargement des données

In [2]:
data = pd.read_csv("nyc_taxi.csv").iloc[:, 1:]
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
data['pickup_datetime'] = data['pickup_datetime'].str.slice(0, 16)
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:00+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:00+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:00+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


## Nettoyage des données

In [4]:
# Remove observations with missing values
# Since there are only a few of these, i'm not concerned with imputation
data.dropna(how='any', axis='rows', inplace=True)

# Removing observations with erroneous values
mask = data['pickup_longitude'].between(-75, -73)
mask &= data['dropoff_longitude'].between(-75, -73)
mask &= data['pickup_latitude'].between(40, 42)
mask &= data['dropoff_latitude'].between(40, 42)
mask &= data['passenger_count'].between(0, 8)
mask &= data['fare_amount'].between(0, 250)

data = data[mask]

## Feature Engineering

In [5]:
# TODO : Implémenter la distance de Manhattan (norme L1)
def dist(pickup_lat, pickup_long, dropoff_lat, dropoff_long):  
    distance = abs(pickup_lat - dropoff_lat) + abs(pickup_long - dropoff_long)
    return distance

In [6]:
def transform(data):
    # Extract date attributes and then drop the pickup_datetime column
    data['hour'] = data['pickup_datetime'].dt.hour
    data['day'] = data['pickup_datetime'].dt.day
    data['month'] = data['pickup_datetime'].dt.month
    data['year'] = data['pickup_datetime'].dt.year
    data = data.drop('pickup_datetime', axis=1)

    # Distances to nearby airports, and city center
    # By reporting distances to these points, the model can somewhat triangulate other locations of interest
    nyc = (-74.0063889, 40.7141667)
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    data['distance_to_center'] = dist(nyc[1], nyc[0],
                                      data['pickup_latitude'], data['pickup_longitude'])
    data['pickup_distance_to_jfk'] = dist(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = dist(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = dist(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = dist(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = dist(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = dist(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    data['long_dist'] = data['pickup_longitude'] - data['dropoff_longitude']
    data['lat_dist'] = data['pickup_latitude'] - data['dropoff_latitude']
    
    data['dist'] = dist(data['pickup_latitude'], data['pickup_longitude'],
                        data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data


dataset = transform(data)
dataset = dataset.drop(['key'], axis=1)
dataset = dataset.sample(10000)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.drop('fare_amount', axis=1),
                                                    dataset['fare_amount'], test_size=0.25)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [8]:
dataset.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,year,distance_to_center,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,long_dist,lat_dist,dist
199116,8.0,-73.962650,40.772976,-73.980375,40.770214,1,7,9,11,2012,0.102548,0.309237,0.324200,0.295326,0.274839,0.095626,0.110589,0.017725,0.002762,0.020487
744868,14.9,-73.996992,40.725480,-73.982674,40.771382,1,13,27,5,2011,0.020710,0.296083,0.327667,0.213488,0.273708,0.171512,0.114056,-0.014318,-0.045902,0.060220
953231,7.0,-73.944727,40.779820,-73.958663,40.773103,2,11,29,12,2014,0.127315,0.298158,0.305377,0.320093,0.299440,0.084547,0.091766,0.013936,0.006717,0.020653
155062,16.5,-73.987185,40.766412,-73.986097,40.730545,2,23,7,11,2014,0.071449,0.327208,0.290253,0.264227,0.229448,0.120773,0.155552,-0.001088,0.035867,0.036955
508406,7.0,-73.961470,40.764682,-73.976637,40.751807,1,8,10,11,2013,0.095434,0.299763,0.302055,0.288212,0.260170,0.096788,0.124830,0.015167,0.012875,0.028042


## Optimisation bayésienne des hyper-paramètres

### Minimisation de la fonction de perte

In [15]:
def xgb_evaluate(max_depth, gamma, colsample_bytree, nrounds):
    params = {'eval_metric': 'rmse',
              'objective':'reg:squarederror',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=int(nrounds), nfold=3, metrics=['rmse'])    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -cv_result['test-rmse-mean'].iloc[-1]

In [16]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             'nrounds': (100, 1000)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=1, n_iter=25, acq='ei')

|   iter    |  target   | colsam... |   gamma   | max_depth |  nrounds  |
-------------------------------------------------------------------------
|  1        | -4.546    |  0.845    |  0.013    |  3.019    |  442.2    |
|  2        | -4.506    |  0.695    |  0.3434   |  3.757    |  537.9    |
|  3        | -4.47     |  0.8022   |  0.03263  |  3.653    |  537.4    |
|  4        | -4.545    |  0.7906   |  0.03847  |  4.074    |  537.4    |
|  5        | -4.491    |  0.8732   |  0.05692  |  5.267    |  604.2    |
|  6        | -4.473    |  0.6162   |  0.1387   |  5.304    |  119.1    |
|  7        | -4.539    |  0.554    |  0.801    |  5.327    |  300.1    |
|  8        | -4.55     |  0.3726   |  0.5522   |  6.352    |  978.7    |
|  9        | -4.525    |  0.6665   |  0.5402   |  5.671    |  954.3    |
|  10       | -4.604    |  0.5012   |  0.7046   |  6.541    |  890.1    |
|  11       | -4.538    |  0.8629   |  0.6338   |  3.532    |  550.0    |
|  12       | -4.644    |  0.7058   | 

In [17]:
xgb_bo.max

{'target': -4.444428867341144,
 'params': {'colsample_bytree': 0.6130475957612771,
  'gamma': 0.0020405259866876913,
  'max_depth': 3.1842032442804213,
  'nrounds': 537.7334906036699}}

## Entraînement du modèle optimisé

In [ ]:
# TODO : Entraîner un modèle avec le jeu d'hyper-paramètres optimal

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score

print(r2_score(y_test, model.predict(X_test)))